# Préparation des données pour l’API + Packaging MLflow 

## Objectif

Cette étape prépare un dataset exploitable en production pour alimenter l’API de scoring (Gradio) et la base SQLite.

## Chargement des données de production simulées

Nous utilisons `test_final.csv` (dataset Kaggle sans variable TARGET) comme proxy de données réelles en production.

Ce fichier représente des demandes clients pour lesquelles on souhaite produire un score de défaut via l’API.

In [1]:
from pathlib import Path
import pandas as pd
import sys
CWD = Path.cwd()
PROJECT_ROOT = CWD.parent.parent
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"
sys.path.append(str(PROJECT_ROOT))
df = pd.read_csv(DATA_PROCESSED / "train.csv")


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\yoann\\Documents\\open classrooms\\projet 8\\livrables\\pret a dépenser\\data\\processed\\train.csv'

## Création du dataset API-ready

Nous construisons un dataset `X_api.csv` contenant :
- `SK_ID_CURR` (identifiant client),
- les 125 features retenues.

Ce fichier sera ensuite utilisé pour alimenter une base SQLite locale afin de simuler le stockage des données en production.

La colonne **SK_ID_CURR** est indispensable pour pouvoir interroger un client spécifique via l’API.

In [ ]:

FEATURE_REDUCTION_DIR = PROJECT_ROOT / "reports" / "feature_reduction"
FEATURE_SET_NAME = "top125_nocorr"
kept_file = FEATURE_REDUCTION_DIR / f"kept_features_{FEATURE_SET_NAME}.txt"

kept_features = [
    l.strip()
    for l in kept_file.read_text(encoding="utf-8").splitlines()
    if l.strip()
]
kept_features = [c for c in kept_features if c in df.columns]
if len(kept_features) == 0:
    raise ValueError(f"kept_features vide -> vérifie le contenu de: {kept_file}")


cols = ["SK_ID_CURR"] + kept_features
missing = [c for c in cols if c not in df.columns]
if missing:
    raise ValueError(f"Colonnes manquantes dans test_final.csv: {missing[:10]}")

## Export du fichier `X_api.csv`

Le fichier est exporté dans `app/asset/`.

Il constitue une source standardisée pour :
- initialiser la base SQLite,
- simuler des appels API,
- générer des données de monitoring (logs de prédiction, scores, latence, drift).

In [ ]:


X_api = df[cols].copy()
out_path = DATA_PROCESSED/ "X_api.csv"
X_api.to_csv(out_path, index=False)

print("OK:", out_path, X_api.shape)

In [ ]:
import json
row = X_api.iloc[3].to_dict()

# Remplacer les NaN par None (=> null en JSON)
row = {k: (None if pd.isna(v) else v) for k, v in row.items()}

# Export JSON formaté
json_text = json.dumps(row, indent=2)

print(json_text)

In [ ]:
import pandas as pd

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

X_api.describe().T

Vérification des bornes pour les chiffres numériques

In [ ]:
desc = X_api.describe().T  # (rows=features)

# bornes simples à partir du describe (min/max observés)
bounds = desc[["min", "max", "mean", "std"]].copy()
bounds

## Récuperer le modèle + artifacts 

In [ ]:
import os
import shutil

RUN_ID = "a4dc7831df1b42a4a363af56bc86a775"

SRC_RUN_DIR = PROJECT_ROOT / "artifacts" / RUN_ID/"artifacts"
DEST_DIR = PROJECT_ROOT/"app"/"assets"

os.makedirs(DEST_DIR, exist_ok=True)

# Copier le dossier model
shutil.copytree(
    f"{SRC_RUN_DIR}/model",
    f"{DEST_DIR}/model",
    dirs_exist_ok=True
)

# Copier le dossier api_artifacts
shutil.copytree(
    f"{SRC_RUN_DIR}/api_artifacts",
    f"{DEST_DIR}/api_artifacts",
    dirs_exist_ok=True
)

print("Copie terminée vers app/assets/")